In [11]:
from tinydb import TinyDB
db = TinyDB("../../bestbelarus.json")
len(db.all())

48

In [12]:
places = db.all()
places[1]

{'name': 'Березинский биосферный заповедник',
 'url': 'https://bestbelarus.by/objects/all/berezinskiy-biosfernyy-zapovednik/',
 'image': 'https://bestbelarus.by/upload/resize_cache/iblock/e4f/358_228_2/e4f163feb15963a216e762d316bbb470.jpg',
 'place': {'name': 'Березинский биосферный заповедник',
  'location': 'Витебская обл., Лепельский р-н, д. Домжерицы, ул. Центральная, 3',
  'coords': [54.74439, 28.317604],
  'sights': [{'name': 'Территория',
    'images': ['https://bestbelarus.by/upload/resize_cache/iblock/f23/0_510_2a18e1f56f4eaac9195ab4f8530310a0f/f2325dc2e98b558a78612e7e146b74ab.jpg',
     'https://bestbelarus.by/upload/resize_cache/iblock/f23/0_510_2a18e1f56f4eaac9195ab4f8530310a0f/f2325dc2e98b558a78612e7e146b74ab.jpg',
     'https://bestbelarus.by/upload/resize_cache/iblock/ab7/0_510_2a18e1f56f4eaac9195ab4f8530310a0f/ab75fb7db77c0d452ea8ee1b32529347.jpg',
     'https://bestbelarus.by/upload/resize_cache/iblock/ab7/0_510_2a18e1f56f4eaac9195ab4f8530310a0f/ab75fb7db77c0d452ea8ee1

In [13]:
unique_towns = set()
town_prefixes = [
    'аг.',
    'г. п.',
    'г.п.',
    'г.',
    'д.',
    'а/г'
]

for place in places:
    location = [l.strip() for l in place["place"]["location"].split(",")]
    for i, location_part in enumerate(location):
        for town_prefix in town_prefixes:
            if location_part.startswith(town_prefix):
                town = location_part.replace(town_prefix, "").strip()
                unique_towns.add(town)
                break

In [14]:
preprocessed_places = []

for place in places:
    town, town_index, town_type = None, None, None
    address = None
    location = [l.strip() for l in place["place"]["location"].split(",")]
    for i, location_part in enumerate(location):
        for town_prefix in town_prefixes:
            if location_part.startswith(town_prefix):
                town = location_part.replace(town_prefix, "").strip()
                town_index = i
                town_type = town_prefix
                break
        if town is not None:
            break
        if location_part in unique_towns:
            town = location_part
            town_index = i
            town_type = None
            break

    if place['name'] != town:
        place['town'] = town
        if town_index is not None:
            address = location[town_index+1:]
    location = location[:town_index]
    preprocessed_places.append({
        "name": place["name"].strip(),
        "coords": place["place"]["coords"],
        "location": location,
        "address": address,
        "town": town,
        "town_type": town_type,
        "image": place["image"],
        "url": {
            'href': place["url"],
            'text': place['title'].strip()
        },
        "sights": place["place"]["sights"]
    })
preprocessed_places[0]["sights"][0]

{'name': 'Территория',
 'images': ['https://bestbelarus.by/upload/resize_cache/iblock/b21/0_510_2a18e1f56f4eaac9195ab4f8530310a0f/b21cf3ae179fa09bee35abc7c1bf569a.jpg',
  'https://bestbelarus.by/upload/resize_cache/iblock/b21/0_510_2a18e1f56f4eaac9195ab4f8530310a0f/b21cf3ae179fa09bee35abc7c1bf569a.jpg',
  'https://bestbelarus.by/upload/resize_cache/iblock/b5d/0_510_2a18e1f56f4eaac9195ab4f8530310a0f/b5d5c50e6d35f6a310db52f4c3dfeb69.jpg',
  'https://bestbelarus.by/upload/resize_cache/iblock/b5d/0_510_2a18e1f56f4eaac9195ab4f8530310a0f/b5d5c50e6d35f6a310db52f4c3dfeb69.jpg',
  'https://bestbelarus.by/upload/resize_cache/iblock/e8b/0_510_2a18e1f56f4eaac9195ab4f8530310a0f/e8b858a67b3cfb09012a7f4d60777b14.jpg',
  'https://bestbelarus.by/upload/resize_cache/iblock/e8b/0_510_2a18e1f56f4eaac9195ab4f8530310a0f/e8b858a67b3cfb09012a7f4d60777b14.jpg',
  'https://bestbelarus.by/upload/resize_cache/iblock/cef/0_510_2a18e1f56f4eaac9195ab4f8530310a0f/cefc5ee156756aed4b469ef11d436740.jpg',
  'https://best

In [15]:
import json

with open("bestbelarus_processed.json", "w", encoding="utf-8") as f:
    json.dump(preprocessed_places, f, ensure_ascii=False, indent=2)